# Configuración

In [10]:
# Configuración + Librerias

!pip install -Uq wandb

#librerias
import torchvision
import torchvision.transforms as transforms
from torchvision.transforms import v2
from sklearn.model_selection import train_test_split
import torch.utils.data
import numpy as np
import wandb
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

#login de wandb
wandb.login()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

torch.manual_seed(7)
if torch.cuda.is_available():
    torch.cuda.manual_seed(7)


wandb: Currently logged in as: rochigonzalezcc. Use `wandb login --relogin` to force relogin




# Ejercicio 8




### Parámetros

In [11]:
#Parametros
batch_size = 64
learning_rate = 0.0001
momentum =.9
epochs = 30
project_name = "clasificación_img"

### Transformaciones

In [12]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(0, 1)
])

val_test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0, 1)
])

### Carga de datos

In [13]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)

targets_ = trainset.targets
train_idx, val_idx = train_test_split(np.arange(len(targets_)), test_size=0.2, stratify=targets_)
train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
val_sampler = torch.utils.data.SubsetRandomSampler(val_idx)

trainloader = torch.utils.data.DataLoader(trainset, sampler=train_sampler,batch_size=batch_size, num_workers=2)
valloader = torch.utils.data.DataLoader(trainset, sampler=val_sampler,batch_size=batch_size, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=val_test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### Red

In [14]:
class red_final(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(256)
        self.conv2 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(512)
        self.conv3 = nn.Conv2d(in_channels=512, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.fc1 = nn.Linear(128 * 4 * 4, 128)
        self.dropout = nn.Dropout(0.2)  # Capa de dropout con probabilidad 0.2
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = nn.GELU()(self.bn1(self.conv1(x)))
        x = nn.MaxPool2d(kernel_size=2, stride=2)(x)
        x = nn.GELU()(self.bn2(self.conv2(x)))
        x = nn.MaxPool2d(kernel_size=2, stride=2)(x)
        x = nn.GELU()(self.bn3(self.conv3(x)))
        x = nn.MaxPool2d(kernel_size=2, stride=2)(x)
        x = torch.flatten(x, 1)
        x = nn.GELU()(self.fc1(x))
        x = self.dropout(x)  # Aplicar dropout a la salida de la primera capa lineal
        x = self.fc2(x)
        return x

### Modelo

In [15]:
experiment_name = "red_final"
net = red_final()
net.to(device)

red_final(
  (conv1): Conv2d(3, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(512, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=2048, out_features=128, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

### Loss y Optimizador

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

### Entrenamiento

In [17]:
wandb.init(
    project= project_name,
    entity = 'tp3-td6',
    name = experiment_name,

    # Hiperparametros
    config={
        "learning_rate": learning_rate,
        "momentum": momentum,
        "batch_size": batch_size,
        "epochs": epochs,
    }
)

start = time.time()

val_accuracies = []
train_losses = []
val_losses = []
best_val_accuracy = 0.0
best_epoch = 0

# loop over the dataset multiple times
for epoch in range(epochs):

    running_loss = 0.0
    train_correct = 0
    total = 0

    for i, data in enumerate(trainloader, 0):

        # Se obtinen los imputs (imagenes) y las labels
        inputs = data[0].to(device)
        labels = data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')


        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        train_correct += (predicted == labels).sum().item()


    # End of test section
    # Val section
    train_accuracy = 100 * train_correct / total
    running_loss = running_loss/total

    val_correct = 0
    total =0
    val_loss =0

    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in valloader:
            images, labels = data[0].to(device), data[1].to(device)
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
            val_loss += criterion(outputs, labels).item()

    # End of test section

    val_accuracy = 100 * val_correct / total
    val_loss = val_loss / total

    val_accuracies.append(val_accuracy)

    train_losses.append(running_loss / total)
    val_losses.append(val_loss / total)

    if val_accuracy > best_val_accuracy:
        best_epoch = epoch
        best_val_accuracy = val_accuracy


    print(f"Guardando el modelo para la época {best_epoch}.")
    torch.save(net.state_dict(), f'{experiment_name}.pth')

    print('Epoch: {}/{}..'.format(epoch+1, epochs),
          'Training loss: {:.3f}..'.format(running_loss / total),
          'Val loss: {:.3f}..'.format(val_loss/ total),
          'Val accuracy: {:.3f}'.format(val_accuracy))

    wandb.log({ "train_accuracy": train_accuracy,
               "val_accuracy": val_accuracy,
                "train_loss": running_loss,
                "val_loss": val_loss,
                "best_val_accuracy": best_val_accuracy,
                "best_epoch": best_epoch })


end = time.time()
train_time = round(end - start)

print(f'Training time: {str(train_time)} seconds.')

wandb.log({'train_time': train_time})


print('Finished Training')





[1,   200] loss: 0.156
[1,   400] loss: 0.278
[1,   600] loss: 0.386
Guardando el modelo para la época 0.
Epoch: 1/30.. Training loss: 0.000.. Val loss: 0.000.. Val accuracy: 64.510
[2,   200] loss: 0.098
[2,   400] loss: 0.188
[2,   600] loss: 0.276
Guardando el modelo para la época 1.
Epoch: 2/30.. Training loss: 0.000.. Val loss: 0.000.. Val accuracy: 69.670
[3,   200] loss: 0.080
[3,   400] loss: 0.160
[3,   600] loss: 0.238
Guardando el modelo para la época 2.
Epoch: 3/30.. Training loss: 0.000.. Val loss: 0.000.. Val accuracy: 73.310
[4,   200] loss: 0.071
[4,   400] loss: 0.142
[4,   600] loss: 0.211
Guardando el modelo para la época 3.
Epoch: 4/30.. Training loss: 0.000.. Val loss: 0.000.. Val accuracy: 75.230
[5,   200] loss: 0.064
[5,   400] loss: 0.129
[5,   600] loss: 0.193
Guardando el modelo para la época 4.
Epoch: 5/30.. Training loss: 0.000.. Val loss: 0.000.. Val accuracy: 75.950
[6,   200] loss: 0.059
[6,   400] loss: 0.117
[6,   600] loss: 0.176
Guardando el modelo p

In [18]:
wandb.finish()

best_epoch,▁▁▂▂▂▂▃▃▃▃▃▄▅▅▅▅▆▆▆▆▆▆▆▆██████
best_val_accuracy,▁▃▅▅▆▆▆▆▇▇▇▇▇▇████████████████
train_accuracy,▁▃▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇██████████
train_loss,█▆▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
train_time,▁
val_accuracy,▁▃▅▅▆▆▆▆▇▇▇▇▇▇██████▇█▇█████▇█
val_loss,█▆▄▃▃▃▂▂▁▂▂▁▁▁▁▁▁▁▂▂▂▂▃▂▂▂▃▃▄▃
best_epoch,24
best_val_accuracy,82.06
train_accuracy,97.53
train_loss,0.0012


### Guardado del modelo final

Ejemplo de guardar el modelo. Sin embargo lo deberiamos guardar para el que mejor dio en validation.



In [26]:
PATH = './{}.pth'.format(experiment_name)
torch.save(net.state_dict(), PATH)

### Cargamos el modelo (el mejor) y evaluamos en un ejemplo.

In [28]:
dataiter = iter(testloader)
data = next(dataiter)
test_images, test_labels = data

In [30]:
# net = red_final()
# net.load_state_dict(torch.load(PATH))
# device = 'cpu'

test_images, test_labels = data[0].to(device), data[1].to(device)
outputs = net(test_images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(10)))

Predicted:  cat   car   car   plane frog  frog  car   frog  cat   truck



Evaluamos en el set de test



In [33]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 80 %


Algo estamos aprendiendo! Si fuese random, seria 10% accuracy (elegir random entre 10 clases).
Algunas clases se comportan peor que otras.


In [35]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: plane is 77.5 %
Accuracy for class: car   is 92.3 %
Accuracy for class: bird  is 68.6 %
Accuracy for class: cat   is 69.4 %
Accuracy for class: deer  is 82.2 %
Accuracy for class: dog   is 73.8 %
Accuracy for class: frog  is 85.2 %
Accuracy for class: horse is 83.8 %
Accuracy for class: ship  is 87.0 %
Accuracy for class: truck is 89.4 %
